In [1]:
import nibabel as nib
import numpy as np
import math
import subprocess
import re


def scanner_to_voxel(coords, affine):
    return np.floor(nib.affines.apply_affine(np.linalg.inv(affine), coords))

def convert_coords(coords, img='/users/zacharydiamandis/Documents/Caltech/ODohertyRotation/boldLFP/resliced_template.nii.gz'):
    # Load the anatomical image
    img = nib.load(img)
    
    # Get the affine transformation matrix
    affine = img.affine

    # Define the scanner space coordinates
    voxel_coords = np.round(scanner_to_voxel(coords, affine)).astype(int)

    return voxel_coords



In [2]:
import os

os.environ["PATH"] += os.pathsep + '/usr/local/fsl/bin'
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"

In [3]:
import pandas as pd
import subprocess

# Read in the MNI locations
mniLocations = pd.read_csv('MNI_electrodePos.csv', header = 3)

# Select the columns that end with "micro" and the "Patient ID" column
columns_to_keep = mniLocations.filter(regex='micro$|^Patient ID$')

# Specify the patient IDs to keep
patient_ids = ['P71CS', 'P70CS', 'P70CS', 'P69CS', 'P69CS', 'P68CS', 'P67CS', 'P67CS', 'P65CS', 'P64CS', 'P63CS', 'P62CS', 'P61CS', 'P60CS']

# Select the rows with the specified patient IDs
microLocations = columns_to_keep[mniLocations['Patient ID'].isin(patient_ids)]

# Extract brain regions
regions = microLocations.columns[1:]

# List of subjects
patients = microLocations['Patient ID']

# Iterate through regions
for region in regions:
    
    # Get coordinates of all subjects for that region
    regionCoords = microLocations[region]
    
    # Number of subjects for that region 
    N = len(regionCoords)
    
    acc_bin_file_list = []
    # Iterate through subjects in region
    for i in range(N):
        patient = patients.iloc[i]
        coord_string = regionCoords.iloc[i]
        if pd.isna(coord_string):
            print("No Coordinates found for '{0}' in '{1}'".format(patient, region))
            continue
        coordsMNI = [float(x) for x in re.findall(r'-?\d+(?:\.\d+)?', coord_string)]
        if len(coordsMNI) != 3:
            print("Incorrect number of coordinates found for '{0}' in '{1}'".format(patient, region))
            continue
        voxel_coords = convert_coords(coordsMNI)
        
        output_folder = "/users/zacharydiamandis/Documents/Caltech/ODohertyRotation/boldLFP/bold/masks/"
        ACCpoint_file = output_folder + region + "_ACCpoint_" + patient
        ACCsphere_file = output_folder + region + "_ACCsphere_" + patient
        ACCsphere_bin_file = output_folder + region + "_ACCsphere_bin_" + patient
        acc_bin_file_list.append(ACCsphere_bin_file)

        fslmaths_command_1 = "fslmaths /users/zacharydiamandis/Documents/Caltech/ODohertyRotation/boldLFP/resliced_template.nii.gz -mul 0 -add 1 -roi {0} 1 {1} 1 {2} 1 0 1 {3} -odt float".format(voxel_coords[0], voxel_coords[1], voxel_coords[2], ACCpoint_file)
        fslmaths_command_2 = "fslmaths {0} -kernel sphere 8 -fmean {1} -odt float".format(ACCpoint_file, ACCsphere_file)
        fslmaths_command_3 = "fslmaths {0} -bin {1}".format(ACCsphere_file, ACCsphere_bin_file)

        subprocess.run(fslmaths_command_1, shell=True, check=True)
        subprocess.run(fslmaths_command_2, shell=True, check=True)
        subprocess.run(fslmaths_command_3, shell=True, check=True)
    
    #union_file = output_folder + region + "_union_of_all_subjects.nii.gz"
    union_output_folder = "/users/zacharydiamandis/Documents/Caltech/ODohertyRotation/boldLFP/bold/union/"

    command = 'fslmaths'
    for i, file_path in enumerate(acc_bin_file_list):
        command += ' {0}'.format(file_path)
        if i < len(acc_bin_file_list) - 1:
            command += ' -add'
    command += ' -bin {0}{1}_union_of_all_subjects.nii.gz'.format(union_output_folder, region)


    fslmaths_command_4 = command

    subprocess.run(fslmaths_command_4, shell=True, check=True)



        
        
    


No Coordinates found for 'P62CS' in 'LAmicro'
No Coordinates found for 'P64CS' in 'RAmicro'
No Coordinates found for 'P62CS' in 'LHFmicro'
No Coordinates found for 'P64CS' in 'RHFmicro'
No Coordinates found for 'P62CS' in 'LACCmicro'
No Coordinates found for 'P62CS' in 'LSMAmicro'
No Coordinates found for 'P62CS' in 'LOFCmicro'


In [4]:
coord_string

'2.48,33.28,-20.24'

In [5]:
# region = 'LAmicro'
# regionCoords = microLocations[region]
    
# # Number of subjects for that region 
# N = len(regionCoords)

# acc_bin_file_list = []
# # Iterate through subjects in region
# for i in range(N):
#     patient = patients.iloc[i]
#     coord_string = regionCoords.iloc[i]
#     if pd.isna(coord_string):
#         print("No Coordinates found for '{0}' in '{1}'".format(patient, region))
#         continue
#     coordsMNI = [float(x) for x in coord_string.split(',')]
#     voxel_coords = convert_coords(coordsMNI)

#     output_folder = "/users/zacharydiamandis/Documents/Caltech/ODohertyRotation/boldLFP/bold/masks/"
#     ACCpoint_file = output_folder + "ACCpoint_" + patient
#     ACCsphere_file = output_folder + "ACCsphere_" + patient
#     ACCsphere_bin_file = output_folder + "ACCsphere_bin_" + patient
#     acc_bin_file_list.append(ACCsphere_bin_file)

#     fslmaths_command_1 = "fslmaths /users/zacharydiamandis/Documents/Caltech/ODohertyRotation/boldLFP/resliced_template.nii.gz -mul 0 -add 1 -roi {0} 1 {1} 1 {2} 1 0 1 {3} -odt float".format(voxel_coords[0], voxel_coords[1], voxel_coords[2], ACCpoint_file)
#     fslmaths_command_2 = "fslmaths {0} -kernel sphere 8 -fmean {1} -odt float".format(ACCpoint_file, ACCsphere_file)
#     fslmaths_command_3 = "fslmaths {0} -bin {1}".format(ACCsphere_file, ACCsphere_bin_file)

#     subprocess.run(fslmaths_command_1, shell=True, check=True)
#     subprocess.run(fslmaths_command_2, shell=True, check=True)
#     subprocess.run(fslmaths_command_3, shell=True, check=True)

# union_file = output_folder + region + "_union_of_all_subjects.nii.gz"


In [6]:
# union_file = output_folder + region + "_union_of_all_subjects.nii.gz"
# union_output_folder = output_folder + '/union/'

# command = 'fslmaths'
# for i, file_path in enumerate(acc_bin_file_list):
#     command += ' {0}'.format(file_path)
#     if i < len(acc_bin_file_list) - 1:
#         command += ' -add'
# command += ' -bin {0}union_of_all_subjects.nii.gz'.format(union_output_folder)


# fslmaths_command_4 = command

# subprocess.run(fslmaths_command_4, shell=True, check=True)